In [1]:
import main
import pandas as pd

/Users/jigs/anaconda3/lib/python3.11/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Load the file
df = pd.read_csv("Training_data_uhi_index 2025-02-04.csv")

# Preview
print(df.shape)
print(df.columns)
print(df.head())
print(df.isnull().sum())


(11269, 4)
Index(['Longitude', 'Latitude', 'datetime', 'UHI Index'], dtype='object')
   Longitude   Latitude          datetime  UHI Index
0 -73.919037  40.814292  24-07-2021 15:53   1.034616
1 -73.918978  40.814365  24-07-2021 15:53   1.028125
2 -73.918927  40.814433  24-07-2021 15:53   1.028125
3 -73.918875  40.814500  24-07-2021 15:53   1.025961
4 -73.918827  40.814560  24-07-2021 15:53   1.025961
Longitude    0
Latitude     0
datetime     0
UHI Index    0
dtype: int64


In [3]:
df.rename(columns={
    'Longitude': 'longitude',
    'Latitude': 'latitude',
    'datetime': 'timestamp',
    'UHI Index': 'uhi_index'
}, inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'], format="%d-%m-%Y %H:%M")

In [4]:
# Run the full batch extraction and save the result
features_df = main.batch_extract_features_with_cache(df, chunk_size=500)

# Optionally, combine with the original UHI index
final_df = pd.concat([df.reset_index(drop=True), features_df], axis=1)

# Save for training
final_df.to_csv("uhi_with_features.csv", index=False)


[Fetch] Processing chunk 0...
[Success] Chunk 0 saved to cache.
[Fetch] Processing chunk 1...
[Success] Chunk 1 saved to cache.
[Fetch] Processing chunk 2...
[Success] Chunk 2 saved to cache.
[Fetch] Processing chunk 3...
[Success] Chunk 3 saved to cache.
[Fetch] Processing chunk 4...
[Success] Chunk 4 saved to cache.
[Fetch] Processing chunk 5...
[Success] Chunk 5 saved to cache.
[Fetch] Processing chunk 6...
[Success] Chunk 6 saved to cache.
[Fetch] Processing chunk 7...
[Success] Chunk 7 saved to cache.
[Fetch] Processing chunk 8...
[Success] Chunk 8 saved to cache.
[Fetch] Processing chunk 9...
[Success] Chunk 9 saved to cache.
[Fetch] Processing chunk 10...
[Success] Chunk 10 saved to cache.
[Fetch] Processing chunk 11...
[Success] Chunk 11 saved to cache.
[Fetch] Processing chunk 12...
[Success] Chunk 12 saved to cache.
[Fetch] Processing chunk 13...
[Success] Chunk 13 saved to cache.
[Fetch] Processing chunk 14...
[Success] Chunk 14 saved to cache.
[Fetch] Processing chunk 15...